##Assessment of the length of stay in Saint Lucia

This code chunk loads the data and performs the following cleaning techniques:
    removes instances where there are blank values which are codied as -1
    removes ages more than 100 years
    defines tourist as between 1-365 nights
    streamlines purpose of visit
    logs variables (exepect length of stay and age)
    makes a list of continous and categorical variables
    converts month of stay to a seasonal variable
    creates a tourist type variable

Notes:
    The tourist type variable is defined as St Lucian born if someone was born in Saint Lucia but a resident of the US and US born if they were both born and a resident of the US
    all persons in the file are over the age of 18
    Immigrant density is defined as st lucian immigrants per 100,000



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.families.family import NegativeBinomial
import tkinter as tk
from tkinter import filedialog
from scipy import stats
from docx import Document
from docx.shared import Inches
from io import BytesIO
import statsmodels.discrete.discrete_model as discrete
from statsmodels.regression.mixed_linear_model import MixedLM

def select_file(title, file_types, save=False):
    """Allow user to select a file"""
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    
    try:
        if save:
            file_path = filedialog.asksaveasfilename(
                title=title,
                filetypes=file_types,
                defaultextension=file_types[0][1]
            )
        else:
            file_path = filedialog.askopenfilename(
                title=title,
                filetypes=file_types
            )
    finally:
        root.destroy()
    
    return file_path if file_path else None

# Allow user to select input file
print("Please select the input Excel file...")
file_path = select_file(
    "Select Excel Data File", 
    [("Excel files", "*.xlsx *.xls"), ("All files", "*.*")]
)

if not file_path:
    print("No file selected. Exiting.")
    exit()

# Import the data
print(f"Loading data from: {file_path}")
df = pd.read_excel(file_path, sheet_name="Sheet")

# Setup
pd.set_option('display.max_columns', None)

# Remove instances where tourist type is "SLU born tourist"
if 'tourist_type' in df.columns:
    df = df[df['tourist_type'] != 'SLU born tourist']
    

# Encode categorical variables if not already encoded
categorical_vars = ['sex', 'marital_status', 'employment_status', 'purpose', 'accomd_type', 'us_state','tourist_type','region_census', 'region_climate']
encoded_vars = {}

for var in categorical_vars:
    if var in df.columns:
        # Check if variable is already numeric
        if not pd.api.types.is_numeric_dtype(df[var]):
            new_var = f"{var}_enc"
            df[new_var] = pd.Categorical(df[var]).codes
            encoded_vars[var] = new_var
        else:
            encoded_vars[var] = var

# Set the truncation point for los (assuming truncation at 0)
df['los_trunc'] = df['los'].copy()
df.loc[df['los_trunc'] <= 0, 'los_trunc'] = np.nan

# Check for missing data
print("\nMissing data summary:")
missing_data_summary = df.isnull().sum()
print(missing_data_summary)

print("\nMissing data patterns:")
missing_patterns = df.isnull().sum(axis=1)
missing_patterns_counts = missing_patterns.value_counts().sort_index()
print(missing_patterns_counts)

# Visualize los distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['los_trunc'], discrete=True)
plt.title('Histogram of Length of Stay')
plt.tight_layout()
los_hist_img = BytesIO()
plt.savefig(los_hist_img, format='png')
los_hist_img.seek(0)
plt.close()

# Summarize los by purpose
purpose_stats = None
if 'purpose_enc' in df.columns:
    print("\nLength of stay by purpose:")
    purpose_stats = df.groupby('purpose_enc')['los_trunc'].agg(['count', 'mean', 'median', 'min', 'max', 'std'])
    print(purpose_stats)

# Detailed summary of los_trunc
print("\nDetailed summary of length of stay:")
los_describe = df['los_trunc'].describe(percentiles=[.25, .5, .75, .90, .95, .99])
print(los_describe)

# Based on month travel create a new variable called seasons i.e. winter, spring, summer, fall
# Define seasons based on month_travel if it is a number
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return np.nan

df['season'] = df['month_travel'].apply(lambda x: get_season(x) if pd.notnull(x) and isinstance(x, (int, float)) else np.nan)

#encoding the season variable
if 'season' in df.columns:
    df['season_enc'] = pd.Categorical(df['season']).codes

# Cleaning process
# Step 1: Drop missing datapoints for key variables
key_vars = ['los', 'immigrant_population', 'import_from_slu', 'age', 'climate_distance','economic_distance',
            encoded_vars.get('sex', 'sex_enc'), 
            encoded_vars.get('marital_status', 'marital_status_enc'), 
            encoded_vars.get('employment_status', 'employment_status_enc'), 
            'distance_miles', 
            encoded_vars.get('purpose', 'purpose_enc'), 
            encoded_vars.get('accomd_type', 'accomd_type_enc'), 
            encoded_vars.get('tourist_type', 'tourist_type_enc'),
            encoded_vars.get('region_census', 'region_census_enc'),
            encoded_vars.get('region_climate', 'region_climate_enc'),
            'state_percapita_income', 'state_unemployment','immigrant_density', 'season_enc']
           
             



#Now that we have the encoded variables, we need to remove blank and -1 values
# Remove rows where sex_enc is -1
df = df[df['sex_enc'] != -1]
# Remove rows where age > 100
df = df[df['age'] <= 100]  
# Remove rows where marital_status_enc is -1
df= df[df['marital_status_enc'] != -1]

#Remove rows where employment_status_enc is -1
df = df[df['employment_status_enc'] != -1]

# Count missing values per row for key variables
df['missing'] = df[key_vars].isnull().sum(axis=1)
print("\nNumber of missing values per observation:")
missing_values_count = df['missing'].value_counts().sort_index()
print(missing_values_count)

# Drop observations with missing values in key variables
df_clean = df[df['missing'] == 0].drop('missing', axis=1)
print(f"\nRemaining observations after dropping missing values: {len(df_clean)}")

# Step 2: Drop outliers in length of stay
#02 June 2025 edit removed the windorization of los_trunc
#los_p95 = np.percentile(df_clean['los_trunc'].dropna(), 95)
df_clean['los_capped'] = df_clean['los_trunc'].copy()
#df_clean.loc[df_clean['los_capped'] > los_p95, 'los_capped'] = los_p95

#df_clean = df_clean[df_clean['los_trunc'] <= los_p95]
print(f"After filtering to 95th percentile, remaining observations: {len(df_clean)}")

# Remove any instance of los_capped that is less than 2. Prior to this, over 1000 persons had stays less than 1 including honeymooners. This appears to be a data entry error.
#02 June 2025 edit use the tourism definition of length of stay where its more than 1 day and less than 366 days

df_clean = df_clean[(df_clean['los_capped'] > 1) & (df_clean['los_capped'] < 366)]  # Keep stays more than 1 and less than 366 days

#remove instances where sex_enc is -1

#30 June 2025
#remove  if us_state enc is 45 or 19 (US virgin islands or Mon repos)

# Visualize the capped los distribution
plt.figure(figsize=(10, 6))
sns.histplot(df_clean['los_capped'], discrete=True)
plt.title('Histogram of Capped Length of Stay')
plt.tight_layout()
los_capped_img = BytesIO()
plt.savefig(los_capped_img, format='png')
los_capped_img.seek(0)
plt.close()

# Step 3: Clean up the purpose of trip column
# Create a new simplified purpose variable
purpose_mapping = {
    0: 1,  # BUSINESS/MEETING -> Business
    1: 1,  # CONVENTION -> Business
    2: 1,  # CREW -> Business
    3: 5,  # CRICKET -> Other
    4: 2,  # EVENT -> Events
    5: 2,  # EVENTS -> Events
    6: 4,  # HONEYMOON -> Pleasure
    7: 5,  # INTRANSIT PASSENGER -> Other
    8: 5,  # OTHER -> Other
    9: 4,  # PLEASURE/HOLIDAY/VACATION -> Pleasure
    10: 5, # RESIDENT -> Other
    11: 2, # SAINT LUCIA CARNIVAL -> Events
    12: 2, # SAINT LUCIA JAZZ AND ARTS FESTIVAL -> Events
    13: 5, # SPORTS -> Other
    14: 5, # STUDY -> Other
    15: 5, # VISITING FRIENDS/RELATIVES -> Other
    16: 3, # WEDDING -> Wedding
    -1: 5, # Missing/Unknown -> Other
}

purpose_labels = {
    1: "Business",
    2: "Events", 
    3: "Wedding",
    4: "Pleasure",
    5: "Other"
}

# Add the simplified purpose variable
purpose_enc_col = encoded_vars.get('purpose', 'purpose_enc')
df_clean['purpose_simple'] = df_clean[purpose_enc_col].map(purpose_mapping)

# Check the new variable
print("\nPurpose simple distribution:")
purpose_counts = df_clean['purpose_simple'].value_counts().sort_index()
purpose_distribution = []
for code, count in purpose_counts.items():
    purpose_line = f"{code} ({purpose_labels.get(code, 'Unknown')}): {count}"
    purpose_distribution.append(purpose_line)
    print(purpose_line)

# Create a Word document for output
doc = Document()
doc.add_heading('Multilevel Truncated Negative Binomial Regression for Length of Stay Analysis', 0)
doc.add_heading('Data Preparation and Cleaning', level=1)



# Add Length of Stay histogram
doc.add_paragraph('\n')
doc.add_heading('Length of Stay Distribution', level=2)
doc.add_picture(los_hist_img, width=Inches(6))
doc.add_paragraph('Figure 1: Histogram of Length of Stay (Before Capping)')

# Add Capped LOS histogram
doc.add_paragraph('\n')
doc.add_heading('Capped Length of Stay Distribution', level=2)
doc.add_picture(los_capped_img, width=Inches(6))
doc.add_paragraph('Figure 2: Histogram of Length of Stay (After Capping at 95th Percentile)')

# Add LOS summary statistics
doc.add_paragraph('\n')
doc.add_heading('Length of Stay Summary Statistics', level=2)
los_stats_table = doc.add_table(rows=len(los_describe)+1, cols=2)
los_stats_table.style = 'Table Grid'
los_stats_table.cell(0, 0).text = 'Statistic'
los_stats_table.cell(0, 1).text = 'Value'
for i, (stat, value) in enumerate(los_describe.items(), 1):
    los_stats_table.cell(i, 0).text = str(stat)
    los_stats_table.cell(i, 1).text = f"{value:.4f}" if isinstance(value, (int, float)) else str(value)

# Add Purpose distribution
doc.add_paragraph('\n')
doc.add_heading('Purpose of Visit Distribution', level=2)
purpose_table = doc.add_table(rows=len(purpose_distribution)+1, cols=1)
purpose_table.style = 'Table Grid'
purpose_table.cell(0, 0).text = 'Purpose Category'
for i, purpose_text in enumerate(purpose_distribution, 1):
    purpose_table.cell(i, 0).text = purpose_text

# Log continuous variables
# Convert columns to numeric before log transformation
for col in ['immigrant_population', 'import_from_slu', 'distance_miles', 'state_percapita_income']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

#df_clean['age'] = np.log1p(df_clean['age'])
df_clean['immigrant_population_log'] = np.log1p(df_clean['immigrant_population'])
df_clean['distance_miles_log'] = np.log1p(df_clean['distance_miles'])
df_clean['state_percapita_income_log'] = np.log1p(df_clean['state_percapita_income'])
df_clean['import_from_slu_log'] = np.log1p(df_clean['import_from_slu'])









In [10]:
# Add labels for all encoded variables in regression models and 
# add continuous variables to the regression model
print("\nFitting simple negative binomial regression model...")
doc.add_paragraph('\n')
doc.add_heading('Negative Binomial Regression Model', level=1)

# Define continuous variables and create proper formula
continuous_vars = ['immigrant_population_log', 'import_from_slu_log', 'age',  
                   'state_percapita_income_log', 'state_unemployment','immigrant_density','climate_distance','economic_distance']

# Make sure all continuous variables are properly formatted as numeric
for var in continuous_vars:
    if var in df_clean.columns:
        df_clean[var] = pd.to_numeric(df_clean[var], errors='coerce')

# Create formula with continuous variables properly treated
formula_parts = []
for var in continuous_vars:
    if var in df_clean.columns:
        formula_parts.append(var)

# Add categorical variables with proper C() notation
categorical_model_vars = ['sex_enc', 'marital_status_enc', 'employment_status_enc', 
                         'purpose_simple', 'accomd_type_enc', 'season_enc', 'us_state_enc','region_census_enc']

for var in categorical_model_vars:
    if var in df_clean.columns:
        # Use the encoded variable name or the original if available
        var_to_use = var
        formula_parts.append(f"C({var_to_use})")

# Combine into final formula
formula = 'los_capped ~ ' + ' + '.join(formula_parts)
print(f"Model formula: {formula}")

# Add formula to document
doc.add_paragraph(f"Model formula: {formula}")

# Drop rows with missing values in formula variables

formula_vars = ['los_capped'] + continuous_vars + categorical_model_vars
df_clean_nb = df_clean[formula_vars].dropna()
df_clean_nb = df_clean_nb.reset_index(drop=True)
print(f"Number of rows in df_clean_nb after dropping missing values: {len(df_clean_nb)}")

# Add this code chunk after your data preparation and before fitting regression models
# This will ensure all categorical variables have proper labels in regression outputs

import pandas as pd
import numpy as np

def create_labeled_categorical_variables(df_clean_nb):
    """
    Create properly labeled categorical variables for regression analysis
    This function maps encoded values back to meaningful labels
    """
    
    
    
    # Define all the label mappings based on your data structure
    
    # Sex labels (assuming 0=Female, 1=Male based on typical encoding)
    sex_labels = {
        0: 'Female',
        1: 'Male'
    }
    
    # Marital Status labels (you may need to adjust these based on your actual encoding)
    marital_status_labels = {
        0: 'Married',
        1: 'Other', 
        2: 'Single'
        
    }
    
    # Employment Status labels (adjust based on your encoding)
    employment_status_labels = {
        0: 'Employed',
        1: 'Unemployed',
        2: 'Student'
       
    }
    
    # Purpose labels (based on your existing purpose_simple mapping)
    purpose_labels = {
        1: 'Business',
        2: 'Events',
        3: 'Wedding', 
        4: 'Pleasure',
        5: 'Other'
    }
    
    # Accommodation type labels (adjust based on your data)
    accomd_type_labels = {
        0: 'Hotel',
        1: 'Other',
        2: 'Sharing'
       
    }
    
    # Season labels (based on your season encoding)
    season_labels = {
        1: 'Spring',
        2: 'Summer', 
        3: 'Winter',
        4: 'Fall'
    }
    
    # Tourist type labels (if you have this variable)
    tourist_type_labels = {
        0: 'US_Born',
        1: 'SLU_Born',
        2: 'Other'
    }
    
    # Region census labels (you'll need to adjust based on your actual regions)
    region_census_labels = {
        1: 'Northeast',
        2: 'Midwest',
        3: 'South', 
        4: 'West',
        5: 'Other'
    }
    
    # US State labels (create a mapping from encoded values to state names)
    # You may want to create this mapping based on your actual state encoding
    us_state_labels = {
        1: 'Alabama', 2: 'Alaska', 3: 'Arizona', 4: 'Arkansas', 5: 'California', 
        6: 'Colorado', 7: 'Connecticut', 8: 'Delaware', 9: 'Florida', 10: 'Georgia',
        11: 'Hawaii', 12: 'Idaho', 13: 'Illinois', 14: 'Indiana', 15: 'Iowa', 
        16: 'Kansas', 17: 'Kentucky', 18: 'Louisiana', 19: 'MON_REPOS', 20: 'Maine',
        21: 'Maryland', 22: 'Massachusetts', 23: 'Michigan', 24: 'Minnesota', 25: 'Mississippi',
        26: 'Missouri', 27: 'Montana', 28: 'Nebraska', 29: 'Nevada', 30: 'New_Hampshire',
        31: 'New_Jersey', 32: 'New_Mexico', 33: 'New_York', 34: 'North_Carolina', 35: 'North_Dakota',
        36: 'Ohio', 37: 'Oklahoma', 38: 'Oregon', 39: 'Pennsylvania', 40: 'Rhode_Island',
        41: 'South_Carolina', 42: 'South_Dakota', 43: 'Tennessee', 44: 'Texas', 45: 'US_VIRGIN_ISLANDS',
        46: 'Utah', 47: 'Vermont', 48: 'Virginia', 49: 'Washington', 50: 'Washington_DC',
        51: 'West_Virginia', 52: 'Wisconsin', 53: 'Wyoming'
    }
    
    # Replace the encoded columns with labeled versions (same column names)
    
    if 'sex_enc' in df_clean_nb.columns:
        df_clean_nb['sex_enc'] = df_clean_nb['sex_enc'].map(sex_labels).fillna('Unknown')
        df_clean_nb['sex_enc'] = pd.Categorical(df_clean_nb['sex_enc'])
    
    if 'marital_status_enc' in df_clean_nb.columns:
        df_clean_nb['marital_status_enc'] = df_clean_nb['marital_status_enc'].map(marital_status_labels).fillna('Unknown')
        df_clean_nb['marital_status_enc'] = pd.Categorical(df_clean_nb['marital_status_enc'])
    
    if 'employment_status_enc' in df_clean_nb.columns:
        df_clean_nb['employment_status_enc'] = df_clean_nb['employment_status_enc'].map(employment_status_labels).fillna('Unknown')
        df_clean_nb['employment_status_enc'] = pd.Categorical(df_clean_nb['employment_status_enc'])
    
    if 'purpose_simple' in df_clean_nb.columns:
        df_clean_nb['purpose_simple'] = df_clean_nb['purpose_simple'].map(purpose_labels).fillna('Unknown')
        df_clean_nb['purpose_simple'] = pd.Categorical(df_clean_nb['purpose_simple'])
    
    if 'accomd_type_enc' in df_clean_nb.columns:
        df_clean_nb['accomd_type_enc'] = df_clean_nb['accomd_type_enc'].map(accomd_type_labels).fillna('Unknown')
        df_clean_nb['accomd_type_enc'] = pd.Categorical(df_clean_nb['accomd_type_enc'])
    
    if 'season_enc' in df_clean_nb.columns:
        df_clean_nb['season_enc'] = df_clean_nb['season_enc'].map(season_labels).fillna('Unknown')
        df_clean_nb['season_enc'] = pd.Categorical(df_clean_nb['season_enc'])
    
    if 'tourist_type_enc' in df_clean_nb.columns:
        df_clean_nb['tourist_type_enc'] = df_clean_nb['tourist_type_enc'].map(tourist_type_labels).fillna('Unknown')
        df_clean_nb['tourist_type_enc'] = pd.Categorical(df_clean_nb['tourist_type_enc'])
    
    if 'region_census_enc' in df_clean_nb.columns:
        df_clean_nb['region_census_enc'] = df_clean_nb['region_census_enc'].map(region_census_labels).fillna('Unknown')
        df_clean_nb['region_census_enc'] = pd.Categorical(df_clean_nb['region_census_enc'])
    
    if 'us_state_enc' in df_clean_nb.columns:
        df_clean_nb['us_state_enc'] = df_clean_nb['us_state_enc'].map(us_state_labels).fillna('Unknown')
        df_clean_nb['us_state_enc'] = pd.Categorical(df_clean_nb['us_state_enc'])
    
    return df_clean_nb

def print_categorical_mappings(df_clean_nb):
    """
    Print the current categorical variables for verification
    """
    print("Categorical Variables After Labeling:")
    print("=" * 50)
    
    categorical_vars = [
        'sex_enc', 'marital_status_enc', 'employment_status_enc', 
        'purpose_simple', 'accomd_type_enc', 'season_enc', 
        'region_census_enc', 'us_state_enc'
    ]
    
    for var in categorical_vars:
        if var in df_clean_nb.columns:
            print(f"\n{var}:")
            value_counts = df_clean_nb[var].value_counts()
            for value, count in value_counts.items():
                print(f"  {value}: {count}")

# Usage - replace your existing dataframe and formulas:

# Same name dataframe with labeled categorical variables
df_clean_nb = create_labeled_categorical_variables(df_clean_nb)
print_categorical_mappings(df_clean_nb)

# Your existing formulas will now work with labeled variables automatically
# because we replaced the encoded columns with labeled versions using the same names

# Same name formula with labeled variables (your original full formula)
# This will now use the labeled categorical variables instead of numeric codes
print(f"\nOriginal formula (now with labels): {formula}")

# Same name formula simple with labeled variables 
# This will now use the labeled categorical variables instead of numeric codes
#print(f"\nSimple formula (now with labels): {formula_simple}")

# Same name formula simple with no state and labeled variables
# This will now use the labeled categorical variables instead of numeric codes  
#print(f"\nSimple formula without state (now with labels): {formula_simple_nostate}")


Fitting simple negative binomial regression model...
Model formula: los_capped ~ immigrant_population_log + import_from_slu_log + age + state_percapita_income_log + state_unemployment + immigrant_density + climate_distance + economic_distance + C(sex_enc) + C(marital_status_enc) + C(employment_status_enc) + C(purpose_simple) + C(accomd_type_enc) + C(season_enc) + C(us_state_enc) + C(region_census_enc)
Number of rows in df_clean_nb after dropping missing values: 139278
Categorical Variables After Labeling:

sex_enc:
  Female: 76562
  Male: 62716

marital_status_enc:
  Married: 97765
  Single: 38573
  Other: 2940

employment_status_enc:
  Employed: 137720
  Unemployed: 1009
  Student: 549

purpose_simple:
  Pleasure: 122211
  Wedding: 6533
  Other: 6028
  Business: 2472
  Events: 2034

accomd_type_enc:
  Hotel: 115209
  Sharing: 18999
  Other: 5070

season_enc:
  Summer: 39501
  Spring: 37803
  Winter: 34187
  Unknown: 27787

region_census_enc:
  South: 51491
  Midwest: 51110
  Northeas

This code chunk runs three negative binomials. In each case all continous data is in logs form with the exception of length of stay, age and continous variables which are ratio's such as the unemployment rate. 

The first negative binomial runs all variables. In that model we see that the VIF for age, distance and state per capita exceeds 100. We also note that the state per capita variable is insignificant. Marital status and employment status are insignificant. 
 

The second negative binomial attempts to address the issues with the first by removing insignificant and highly correlated variables

The third model is the same as the second with the exception that it removes the categorical variable US state and reestimates the standard errors.



In [11]:
# Fit simple negative binomial regression with continuous variables correctly specified


# Fit negative binomial model with all variables
nb_model = smf.glm(formula=formula, 
                  data=df_clean_nb, 
                  family=sm.families.NegativeBinomial(link=sm.families.links.log()))

try:
    nb_results = nb_model.fit()
    print("\nNegative Binomial Regression Results:")
    summary_text = str(nb_results.summary())
    print(summary_text)
    
    # Add model summary to document
    #doc.add_paragraph('\nModel Summary:')
    summary_paragraph_neg = doc.add_paragraph()
    summary_run_neg = summary_paragraph_neg.add_run(summary_text)
    summary_run_neg.font.name = 'Courier New'  # Use monospace font
    #for line in summary_text.split('\n'):
        #doc.add_paragraph(line)
    
    # Convert coefficients to incident rate ratios (IRR)
    print("\nIncident Rate Ratios (IRR):")
    irr = np.exp(nb_results.params)
    irr_conf = np.exp(nb_results.conf_int())
    irr_df = pd.DataFrame({'IRR': irr, 'Lower CI': irr_conf[0], 'Upper CI': irr_conf[1], 
                          'P-value': nb_results.pvalues})
    print(irr_df)
    
    # Add IRR table to document
    doc.add_paragraph('\n')
    doc.add_heading('Incident Rate Ratios (IRR)', level=2)
    irr_table = doc.add_table(rows=len(irr_df)+1, cols=5)
    irr_table.style = 'Table Grid'
    irr_table.cell(0, 0).text = 'Variable'
    irr_table.cell(0, 1).text = 'IRR'
    irr_table.cell(0, 2).text = 'Lower CI'
    irr_table.cell(0, 3).text = 'Upper CI'
    irr_table.cell(0, 4).text = 'P-value'
    
    for i, (var, row) in enumerate(irr_df.iterrows(), 1):
        irr_table.cell(i, 0).text = str(var)
        irr_table.cell(i, 1).text = f"{row['IRR']:.4f}"
        irr_table.cell(i, 2).text = f"{row['Lower CI']:.4f}"
        irr_table.cell(i, 3).text = f"{row['Upper CI']:.4f}"
        irr_table.cell(i, 4).text = f"{row['P-value']:.4f}"
    
    
    # Predictions and diagnostics
    df_clean_nb['predicted'] = nb_results.predict()
    df_clean_nb['residuals'] = df_clean_nb['los_capped'] - df_clean_nb['predicted']
    
    # Plot residuals
    plt.figure(figsize=(10, 6))
    plt.scatter(df_clean_nb['predicted'], df_clean_nb['residuals'], alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='-')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')
    plt.tight_layout()
    residuals_img = BytesIO()
    plt.savefig(residuals_img, format='png')
    residuals_img.seek(0)
    plt.close()
    
    # Add residuals plot to document
    doc.add_paragraph('\n')
    doc.add_heading('Diagnostics', level=2)
    doc.add_picture(residuals_img, width=Inches(6))
    doc.add_paragraph('Figure 3: Residuals Plot')

    #Add VIF for continuous variables and categorical variables
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    vif_data = pd.DataFrame()
    vif_data["Variable"] = df_clean_nb[continuous_vars + categorical_model_vars].columns
    vif_data["VIF"] = [variance_inflation_factor(df_clean_nb[continuous_vars + categorical_model_vars].values, i) 
                       for i in range(df_clean_nb[continuous_vars + categorical_model_vars].shape[1])]
    print("VIF for continuous and dummy variables:")
    print(vif_data)

    #Add VIF table to document
    doc.add_paragraph('\nVariance Inflation Factor (VIF) for Continuous and Categorical Variables:')    
    vif_table = doc.add_table(rows=len(vif_data)+1, cols=2)
    vif_table.style = 'Table Grid'
    vif_table.cell(0, 0).text = 'Variable'
    vif_table.cell(0, 1).text = 'VIF'
    for i, (var, vif) in enumerate(zip(vif_data["Variable"], vif_data["VIF"]), 1):
        vif_table.cell(i, 0).text = str(var)
        vif_table.cell(i, 1).text = f"{vif:.4f}"
    doc.add_paragraph('VIF values above 10 indicate potential multicollinearity issues.')
#except Exception as e:
    #error_msg = f"Error fitting negative binomial model: {str(e)}"
    #print(error_msg)
    #doc.add_paragraph(error_msg)
    #doc.add_paragraph("The negative binomial regression model failed to converge. This can happen due to " +
                     #"insufficient variation in the data or other model specification issues.")
    

# Fit a negative binomial model with fewer variables

#New continuous variables for the simpler model
    continuous_vars_simple = [ 'import_from_slu_log', 'age',  
                    'state_unemployment','immigrant_density','climate_distance','economic_distance']

#New categorical variables for the simpler model

    categorical_model_vars_simple = ['sex_enc',  
                         'purpose_simple', 'accomd_type_enc', 'season_enc', 'us_state_enc']
    
    categorical_model_vars_simple_nostate= ['sex_enc',  
                         'purpose_simple', 'accomd_type_enc', 'season_enc',  'region_census_enc']
    
#New formula for the simpler model
    formula_parts_simple = []
    for var in continuous_vars_simple:
        if var in df_clean_nb.columns:
            formula_parts_simple.append(var)
    # Add categorical variables with proper C() notation
    for var in categorical_model_vars_simple:
        if var in df_clean_nb.columns:
            # Use the encoded variable name or the original if available
            var_to_use = var
            formula_parts_simple.append(f"C({var_to_use})")
    # Combine into final formula
    formula_simple = 'los_capped ~ ' + ' + '.join(formula_parts_simple)
    print(f"Model formula for simpler model: {formula_simple}")
    # Add formula to document
    doc.add_paragraph(f"Model formula for simpler model: {formula_simple}")

#New formula for the simpler model without the state variable
    formula_parts_simple_nostate = []
    for var in continuous_vars_simple:
        if var in df_clean_nb.columns:
            formula_parts_simple_nostate.append(var)
    # Add categorical variables with proper C() notation
    for var in categorical_model_vars_simple_nostate:
        if var in df_clean_nb.columns:
            # Use the encoded variable name or the original if available
            var_to_use = var
            formula_parts_simple_nostate.append(f"C({var_to_use})")
    # Combine into final formula
    formula_simple_nostate = 'los_capped ~ ' + ' + '.join(formula_parts_simple_nostate)
    print(f"Model formula for simpler model without state: {formula_simple_nostate}")

# Fit negative binomial model with fewer variables
    nb_model_simple = smf.glm(formula=formula_simple, 
                          data=df_clean_nb, 
                          family=sm.families.NegativeBinomial(link=sm.families.links.log()))
    try:
        nb_results_simple = nb_model_simple.fit()
        print("\nNegative Binomial Regression Results (Simpler Model):")
        summary_text_simple = str(nb_results_simple.summary())
        print(summary_text_simple)
        
        # Add model summary to document
        summary_paragraph_neg_simple = doc.add_paragraph()
        summary_run_neg_simple = summary_paragraph_neg_simple.add_run(summary_text_simple)
        summary_run_neg_simple.font.name = 'Courier New'  # Use monospace font
        
        # Convert coefficients to incident rate ratios (IRR)
        print("\nIncident Rate Ratios (IRR) for Simpler Model:")
        irr_simple = np.exp(nb_results_simple.params)
        irr_conf_simple = np.exp(nb_results_simple.conf_int())
        irr_df_simple = pd.DataFrame({'IRR': irr_simple, 'Lower CI': irr_conf_simple[0], 'Upper CI': irr_conf_simple[1], 
                                    'P-value': nb_results_simple.pvalues})
        print(irr_df_simple)
        
        # Add IRR table to document
        doc.add_paragraph('\n')
        doc.add_heading('Incident Rate Ratios (IRR) - Simpler Model', level=2)
        irr_table_simple = doc.add_table(rows=len(irr_df_simple)+1, cols=5)
        irr_table_simple.style = 'Table Grid'
        irr_table_simple.cell(0, 0).text = 'Variable'
        irr_table_simple.cell(0, 1).text = 'IRR'
        irr_table_simple.cell(0, 2).text = 'Lower CI'
        irr_table_simple.cell(0, 3).text = 'Upper CI'
        irr_table_simple.cell(0, 4).text = 'P-value'
        
        for i, (var, row) in enumerate(irr_df_simple.iterrows(), 1):
            irr_table_simple.cell(i, 0).text = str(var)
            irr_table_simple.cell(i, 1).text = f"{row['IRR']:.4f}"
            irr_table_simple.cell(i, 2).text = f"{row['Lower CI']:.4f}"
            irr_table_simple.cell(i, 3).text = f"{row['Upper CI']:.4f}"
            irr_table_simple.cell(i, 4).text = f"{row['P-value']:.4f}"
        # Predictions and diagnostics for simpler model
        df_clean_nb['predicted_simple'] = nb_results_simple.predict()
        df_clean_nb['residuals_simple'] = df_clean_nb['los_capped'] - df_clean_nb['predicted_simple']
        # Plot residuals for simpler model
        plt.figure(figsize=(10, 6))
        plt.scatter(df_clean_nb['predicted_simple'], df_clean_nb['residuals_simple'], alpha=0.5)
        plt.axhline(y=0, color='r', linestyle='-')
        plt.xlabel('Predicted Values (Simpler Model)')
        plt.ylabel('Residuals (Simpler Model)')
        plt.title('Residual Plot (Simpler Model)')
        plt.tight_layout()
        residuals_img_simple = BytesIO()
        plt.savefig(residuals_img_simple, format='png')
        residuals_img_simple.seek(0)
        plt.close()
        # Add residuals plot for simpler model to document
        doc.add_paragraph('\n')
        doc.add_heading('Diagnostics - Simpler Model', level=2)
        doc.add_picture(residuals_img_simple, width=Inches(6))
        doc.add_paragraph('Figure 4: Residuals Plot (Simpler Model)')
        # Add VIF for continuous variables and categorical variables for simpler model
        vif_data_simple = pd.DataFrame()
        vif_data_simple["Variable"] = df_clean_nb[continuous_vars_simple + categorical_model_vars_simple].columns
        vif_data_simple["VIF"] = [variance_inflation_factor(df_clean_nb[continuous_vars_simple + categorical_model_vars_simple].values, i) 
                                for i in range(df_clean_nb[continuous_vars_simple + categorical_model_vars_simple].shape[1])]
        print("VIF for continuous and dummy variables (Simpler Model):")
        print(vif_data_simple)
        # Add VIF table for simpler model to document
        doc.add_paragraph('\nVariance Inflation Factor (VIF) for Continuous and Categorical Variables - Simpler Model:')
        vif_table_simple = doc.add_table(rows=len(vif_data_simple)+1, cols=2)
        vif_table_simple.style = 'Table Grid'
        vif_table_simple.cell(0, 0).text = 'Variable'
        vif_table_simple.cell(0, 1).text = 'VIF'
        for i, (var, vif) in enumerate(zip(vif_data_simple["Variable"], vif_data_simple["VIF"]), 1):
            vif_table_simple.cell(i, 0).text = str(var)
            vif_table_simple.cell(i, 1).text = f"{vif:.4f}"
        doc.add_paragraph('VIF values above 10 indicate potential multicollinearity issues in the simpler model.')

        #start of third model without the state variable
        nb_model_simple = smf.glm(formula=formula_simple_nostate, 
                          data=df_clean_nb, 
                          family=sm.families.NegativeBinomial(link=sm.families.links.log()))
        try:
            nb_results_simple = nb_model_simple.fit()
            print("\nNegative Binomial Regression Results (Simpler Model):")
            summary_text_simple = str(nb_results_simple.summary())
            print(summary_text_simple)
            
            # Add model summary to document
            summary_paragraph_neg_simple = doc.add_paragraph()
            summary_run_neg_simple = summary_paragraph_neg_simple.add_run(summary_text_simple)
            summary_run_neg_simple.font.name = 'Courier New'  # Use monospace font
            
            # Convert coefficients to incident rate ratios (IRR)
            print("\nIncident Rate Ratios (IRR) for Simpler Model:")
            irr_simple = np.exp(nb_results_simple.params)
            irr_conf_simple = np.exp(nb_results_simple.conf_int())
            irr_df_simple = pd.DataFrame({'IRR': irr_simple, 'Lower CI': irr_conf_simple[0], 'Upper CI': irr_conf_simple[1], 
                                        'P-value': nb_results_simple.pvalues})
            print(irr_df_simple)
            
            # Add IRR table to document
            doc.add_paragraph('\n')
            doc.add_heading('Incident Rate Ratios (IRR) - Simpler Model', level=2)
            irr_table_simple = doc.add_table(rows=len(irr_df_simple)+1, cols=5)
            irr_table_simple.style = 'Table Grid'
            irr_table_simple.cell(0, 0).text = 'Variable'
            irr_table_simple.cell(0, 1).text = 'IRR'
            irr_table_simple.cell(0, 2).text = 'Lower CI'
            irr_table_simple.cell(0, 3).text = 'Upper CI'
            irr_table_simple.cell(0, 4).text = 'P-value'
            
            for i, (var, row) in enumerate(irr_df_simple.iterrows(), 1):
                irr_table_simple.cell(i, 0).text = str(var)
                irr_table_simple.cell(i, 1).text = f"{row['IRR']:.4f}"
                irr_table_simple.cell(i, 2).text = f"{row['Lower CI']:.4f}"
                irr_table_simple.cell(i, 3).text = f"{row['Upper CI']:.4f}"
                irr_table_simple.cell(i, 4).text = f"{row['P-value']:.4f}"
            # Predictions and diagnostics for simpler model
            df_clean_nb['predicted_simple'] = nb_results_simple.predict()
            df_clean_nb['residuals_simple'] = df_clean_nb['los_capped'] - df_clean_nb['predicted_simple']
            # Plot residuals for simpler model
            plt.figure(figsize=(10, 6))
            plt.scatter(df_clean_nb['predicted_simple'], df_clean_nb['residuals_simple'], alpha=0.5)
            plt.axhline(y=0, color='r', linestyle='-')
            plt.xlabel('Predicted Values (Simpler Model)')
            plt.ylabel('Residuals (Simpler Model)')
            plt.title('Residual Plot (Simpler Model)')
            plt.tight_layout()
            residuals_img_simple = BytesIO()
            plt.savefig(residuals_img_simple, format='png')
            residuals_img_simple.seek(0)
            plt.close()
            # Add residuals plot for simpler model to document
            doc.add_paragraph('\n')
            doc.add_heading('Diagnostics - Simpler Model', level=2)
            doc.add_picture(residuals_img_simple, width=Inches(6))
            doc.add_paragraph('Figure 4: Residuals Plot (Simpler Model)')
            # Add VIF for continuous variables and categorical variables for simpler model
            vif_data_simple = pd.DataFrame()
            vif_data_simple["Variable"] = df_clean_nb[continuous_vars_simple + categorical_model_vars_simple].columns
            vif_data_simple["VIF"] = [variance_inflation_factor(df_clean_nb[continuous_vars_simple + categorical_model_vars_simple].values, i) 
                                    for i in range(df_clean_nb[continuous_vars_simple + categorical_model_vars_simple].shape[1])]
            print("VIF for continuous and dummy variables (Simpler Model):")
            print(vif_data_simple)
            # Add VIF table for simpler model to document
            doc.add_paragraph('\nVariance Inflation Factor (VIF) for Continuous and Categorical Variables - Simpler Model:')
            vif_table_simple = doc.add_table(rows=len(vif_data_simple)+1, cols=2)
            vif_table_simple.style = 'Table Grid'
            vif_table_simple.cell(0, 0).text = 'Variable'
            vif_table_simple.cell(0, 1).text = 'VIF'
            for i, (var, vif) in enumerate(zip(vif_data_simple["Variable"], vif_data_simple["VIF"]), 1):
                vif_table_simple.cell(i, 0).text = str(var)
                vif_table_simple.cell(i, 1).text = f"{vif:.4f}"
            doc.add_paragraph('VIF values above 10 indicate potential multicollinearity issues in the simpler model.')
        #end of third model
    
        except Exception as e:
            error_msg_simple = f"Error fitting negative binomial model (Simpler Model): {str(e)}"
            print(error_msg_simple)
            doc.add_paragraph(error_msg_simple)
            doc.add_paragraph("The negative binomial regression model with fewer variables failed to converge. " +
                            "This can happen due to insufficient variation in the data or other model specification issues.")
    
    except Exception as e:
        error_msg_simple = f"Error fitting negative binomial model (Simpler Model): {str(e)}"
        print(error_msg_simple)
        doc.add_paragraph(error_msg_simple)
        doc.add_paragraph("The negative binomial regression model with fewer variables failed to converge. " +
                         "This can happen due to insufficient variation in the data or other model specification issues.")
except Exception as e:
    error_msg_simple = f"Error fitting negative binomial model (Simpler Model): {str(e)}"
    print(error_msg_simple)
    doc.add_paragraph(error_msg_simple)
    doc.add_paragraph("The negative binomial regression model with fewer variables failed to converge. " +
                        "This can happen due to insufficient variation in the data or other model specification issues.")       



# End of negative binomial models






/opt/anaconda3/envs/jlslu2025/lib/python3.13/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
/opt/anaconda3/envs/jlslu2025/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "



Negative Binomial Regression Results:
                 Generalized Linear Model Regression Results                  
Dep. Variable:             los_capped   No. Observations:               139278
Model:                            GLM   Df Residuals:                   139213
Model Family:        NegativeBinomial   Df Model:                           64
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.1576e+05
Date:                Mon, 30 Jun 2025   Deviance:                       35883.
Time:                        14:49:19   Pearson chi2:                 2.65e+05
No. Iterations:                     8   Pseudo R-squ. (CS):            0.02774
Covariance Type:            nonrobust                                         
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------

In [ ]:
#Draw a scatter plot between age_log and distance_miles_log
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df_clean_nb['age'], y=df_clean_nb['distance_miles_log'])
plt.title('Scatter Plot of  Age vs Log Distance Miles')
plt.xlabel('Age')
plt.ylabel('Log Distance Miles')
plt.savefig('scatter_plot.png', dpi=150, bbox_inches='tight')
plt.show()
#scatter_img = BytesIO()
#plt.savefig(scatter_img, format='png')
#scatter_img.seek(0)
plt.close()
# Add scatter plot to document
doc.add_paragraph('\n')
doc.add_heading('Scatter Plot of Age vs Log Distance Miles', level=2)
doc.add_picture('scatter_plot.png', width=Inches(6))
doc.add_paragraph('Figure 5: Scatter Plot of Age vs Log Distance Miles')

This component re runs the third model with clustered standard errors around state

In [12]:
# Negative Binomial Model with Clustered Robust Standard Errors

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

def fit_nb_with_clustered_se(df_clean_nb, formula_simple_nostate):
    """
    Fit negative binomial model with robust standard errors clustered by US state
    """
    
    # Fit the model (same as before)
    nb_model_simple = smf.glm(formula=formula_simple_nostate, 
                              data=df_clean_nb, 
                              family=sm.families.NegativeBinomial(link=sm.families.links.log()))
    
    # Fit with clustered robust standard errors
    nb_results_clustered = nb_model_simple.fit(
        cov_type='cluster',
        cov_kwds={'groups': df_clean_nb['us_state_enc']}  # Cluster by state
    )
    
    return nb_results_clustered

def compare_standard_errors(df_clean_nb, formula_simple_nostate):
    """
    Compare regular vs clustered standard errors
    """
    
    # Model setup
    nb_model_simple = smf.glm(formula=formula_simple_nostate, 
                              data=df_clean_nb, 
                              family=sm.families.NegativeBinomial(link=sm.families.links.log()))
    
    # Regular standard errors
    nb_results_regular = nb_model_simple.fit()
    
    # Clustered standard errors  
    nb_results_clustered = nb_model_simple.fit(
        cov_type='cluster',
        cov_kwds={'groups': df_clean_nb['us_state_enc']}
    )
    
    # Other robust SE options you might consider:
    # Heteroskedasticity-robust (White's)
    nb_results_robust = nb_model_simple.fit(cov_type='HC1')
    
    # Newey-West (for time series, if applicable)
    # nb_results_nw = nb_model_simple.fit(cov_type='HAC', cov_kwds={'maxlags': 1})
    
    # Compare results
    comparison_df = pd.DataFrame({
        'Variable': nb_results_regular.params.index,
        'Coefficient': nb_results_regular.params.values,
        'SE_Regular': nb_results_regular.bse.values,
        'SE_Clustered': nb_results_clustered.bse.values,
        'SE_Robust': nb_results_robust.bse.values,
        'P_Regular': nb_results_regular.pvalues.values,
        'P_Clustered': nb_results_clustered.pvalues.values,
        'P_Robust': nb_results_robust.pvalues.values
    })
    
    # Calculate ratio of clustered to regular SE
    comparison_df['SE_Ratio_Clustered'] = comparison_df['SE_Clustered'] / comparison_df['SE_Regular']
    comparison_df['SE_Ratio_Robust'] = comparison_df['SE_Robust'] / comparison_df['SE_Regular']
    
    return nb_results_clustered, comparison_df

def print_clustered_results(results, comparison_df=None):
    """
    Print results with clustered standard errors
    """
    print("Negative Binomial Results with Clustered Standard Errors (by US State)")
    print("=" * 80)
    print(results.summary())
    
    if comparison_df is not None:
        print("\n" + "=" * 80)
        print("COMPARISON OF STANDARD ERRORS")
        print("=" * 80)
        print("SE_Ratio > 1 indicates clustering increased standard errors")
        print("SE_Ratio < 1 indicates clustering decreased standard errors")
        print()
        
        # Format comparison table nicely
        display_df = comparison_df.copy()
        for col in ['Coefficient', 'SE_Regular', 'SE_Clustered', 'SE_Robust']:
            display_df[col] = display_df[col].round(4)
        for col in ['P_Regular', 'P_Clustered', 'P_Robust']:
            display_df[col] = display_df[col].round(4)
        for col in ['SE_Ratio_Clustered', 'SE_Ratio_Robust']:
            display_df[col] = display_df[col].round(3)
            
        print(display_df[['Variable', 'Coefficient', 'SE_Regular', 'SE_Clustered', 
                         'SE_Ratio_Clustered', 'P_Regular', 'P_Clustered']].to_string(index=False))

def calculate_clustered_irr(results):
    """
    Calculate IRR with clustered confidence intervals
    """
    # IRR (Incident Rate Ratios)
    irr = np.exp(results.params)
    
    # Confidence intervals with clustered SEs
    irr_conf = np.exp(results.conf_int())
    
    irr_df = pd.DataFrame({
        'IRR': irr,
        'Lower_CI': irr_conf.iloc[:, 0],
        'Upper_CI': irr_conf.iloc[:, 1],
        'P_value': results.pvalues,
        'Significant': results.pvalues < 0.05
    })
    
    return irr_df



# Additional clustering options if needed:
def alternative_clustering_methods(df_clean_nb, formula_simple_nostate):
    """
    Show alternative clustering approaches
    """
    nb_model = smf.glm(formula=formula_simple_nostate, 
                       data=df_clean_nb, 
                       family=sm.families.NegativeBinomial(link=sm.families.links.log()))
    
    # Method 1: Cluster by state (recommended for your case)
    results_state = nb_model.fit(
        cov_type='cluster',
        cov_kwds={'groups': df_clean_nb['us_state_enc']}
    )
    
    # Method 2: Two-way clustering (if you had both state and time)
    # results_twoway = nb_model.fit(
    #     cov_type='cluster',
    #     cov_kwds={'groups': [df_clean_nb['us_state_enc'], df_clean_nb['year']]}
    # )
    
    # Method 3: Bootstrap standard errors (alternative approach)
    # This is more computationally intensive but doesn't require specific clustering assumptions
    
    return results_state



In [13]:
# Fit model with clustered SEs
nb_results_clustered, comparison_df = compare_standard_errors(df_clean_nb, formula_simple_nostate)


print_clustered_results(nb_results_clustered, comparison_df)

# Calculate IRR with clustered confidence intervals
irr_clustered = calculate_clustered_irr(nb_results_clustered)
print("\\nIncident Rate Ratios with Clustered Standard Errors:")
print(irr_clustered.round(4))

# Add clustered results to document
doc.add_paragraph('\n')
doc.add_heading('Negative Binomial Regression with Clustered Standard Errors', level=1)
summary_paragraph_clustered = doc.add_paragraph()
summary_run_clustered = summary_paragraph_clustered.add_run(str(nb_results_clustered.summary()))
summary_run_clustered.font.name = 'Courier New'  # Use monospace font
# Add IRR table with clustered standard errors
doc.add_paragraph('\n')
doc.add_heading('Incident Rate Ratios (IRR) with Clustered Standard Errors', level=2)
irr_table_clustered = doc.add_table(rows=len(irr_clustered)+1, cols=6)
irr_table_clustered.style = 'Table Grid'
irr_table_clustered.cell(0, 0).text = 'Variable'
irr_table_clustered.cell(0, 1).text = 'IRR'
irr_table_clustered.cell(0, 2).text = 'Lower CI'
irr_table_clustered.cell(0, 3).text = 'Upper CI'
irr_table_clustered.cell(0, 4).text = 'P-value'
irr_table_clustered.cell(0, 5).text = 'Significant'
for i, (var, row) in enumerate(irr_clustered.iterrows(), 1):
    irr_table_clustered.cell(i, 0).text = str(var)
    irr_table_clustered.cell(i, 1).text = f"{row['IRR']:.4f}"
    irr_table_clustered.cell(i, 2).text = f"{row['Lower_CI']:.4f}"
    irr_table_clustered.cell(i, 3).text = f"{row['Upper_CI']:.4f}"
    irr_table_clustered.cell(i, 4).text = f"{row['P_value']:.4f}"
    irr_table_clustered.cell(i, 5).text = 'Yes' if row['Significant'] else 'No'
# Add comparison of standard errors table
doc.add_paragraph('\n')
doc.add_heading('Comparison of Standard Errors', level=2)
comparison_table = doc.add_table(rows=len(comparison_df)+1, cols=7)
comparison_table.style = 'Table Grid'
comparison_table.cell(0, 0).text = 'Variable'
comparison_table.cell(0, 1).text = 'Coefficient'
comparison_table.cell(0, 2).text = 'SE Regular'
comparison_table.cell(0, 3).text = 'SE Clustered'
comparison_table.cell(0, 4).text = 'SE Ratio Clustered'
comparison_table.cell(0, 5).text = 'P Regular'
comparison_table.cell(0, 6).text = 'P Clustered'   
for i, row in comparison_df.iterrows():
    comparison_table.cell(i+1, 0).text = str(row['Variable'])
    comparison_table.cell(i+1, 1).text = f"{row['Coefficient']:.4f}"
    comparison_table.cell(i+1, 2).text = f"{row['SE_Regular']:.4f}"
    comparison_table.cell(i+1, 3).text = f"{row['SE_Clustered']:.4f}"
    comparison_table.cell(i+1, 4).text = f"{row['SE_Ratio_Clustered']:.3f}"
    comparison_table.cell(i+1, 5).text = f"{row['P_Regular']:.4f}"
    comparison_table.cell(i+1, 6).text = f"{row['P_Clustered']:.4f}"
# Tips for interpretation:
"""
INTERPRETATION GUIDE:

1. Clustered SEs account for correlation within states
   - Observations from same state may be more similar
   - This typically INCREASES standard errors
   - More conservative inference (larger p-values)

2. When to use clustered SEs:
   - You have multiple observations per cluster (state)
   - You suspect within-cluster correlation
   - State-level policies might affect all observations from that state

3. Ratio interpretation:
   - SE_Ratio > 1.5: Strong evidence of within-cluster correlation
   - SE_Ratio > 2.0: Very strong clustering effects
   - SE_Ratio < 1.1: Little evidence of clustering

4. Model selection:
   - If clustered SEs are much larger, use them for inference
   - If they're similar to regular SEs, either is fine
   - Always report which type you used
"""

/opt/anaconda3/envs/jlslu2025/lib/python3.13/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
/opt/anaconda3/envs/jlslu2025/lib/python3.13/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


Negative Binomial Results with Clustered Standard Errors (by US State)
                 Generalized Linear Model Regression Results                  
Dep. Variable:             los_capped   No. Observations:               139278
Model:                            GLM   Df Residuals:                   139258
Model Family:        NegativeBinomial   Df Model:                           19
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -4.1591e+05
Date:                Mon, 30 Jun 2025   Deviance:                       36195.
Time:                        14:49:45   Pearson chi2:                 2.74e+05
No. Iterations:                     8   Pseudo R-squ. (CS):            0.02556
Covariance Type:              cluster                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------

"\nINTERPRETATION GUIDE:\n\n1. Clustered SEs account for correlation within states\n   - Observations from same state may be more similar\n   - This typically INCREASES standard errors\n   - More conservative inference (larger p-values)\n\n2. When to use clustered SEs:\n   - You have multiple observations per cluster (state)\n   - You suspect within-cluster correlation\n   - State-level policies might affect all observations from that state\n\n3. Ratio interpretation:\n   - SE_Ratio > 1.5: Strong evidence of within-cluster correlation\n   - SE_Ratio > 2.0: Very strong clustering effects\n   - SE_Ratio < 1.1: Little evidence of clustering\n\n4. Model selection:\n   - If clustered SEs are much larger, use them for inference\n   - If they're similar to regular SEs, either is fine\n   - Always report which type you used\n"

We see a negative relationship between imports from St Lucia and immigrant density and length of stay. In the charts/tables below we attempt to dig a bit deeper into this. 

In [ ]:
#create a table showing top 10 states who import the most from SLU and their LOS capped and number of observations
top_states = df_clean_nb.groupby('us_state_enc').agg({
    'import_from_slu_log': 'sum',
    'los_capped': 'mean',
    'us_state_enc': 'count',
    'immigrant_density': 'mean'
}).rename(columns={'us_state_enc': 'num_observations'}).reset_index()
top_states = top_states.sort_values(by='import_from_slu_log', ascending=False).head(10)
#print the top states table
print("\nTop 10 States Importing from SLU:")
print(top_states)

# Add top states table to document
doc.add_paragraph('\n')
doc.add_heading('Top 10 States Importing from SLU', level=2)
top_states_table = doc.add_table(rows=len(top_states)+1, cols=5)
top_states_table.style = 'Table Grid'
top_states_table.cell(0, 0).text = 'State'
top_states_table.cell(0, 1).text = 'Total Import from SLU (Log)'
top_states_table.cell(0, 2).text = 'Mean Length of Stay (Capped)'
top_states_table.cell(0, 3).text = 'Number of Observations'
top_states_table.cell(0, 4).text = 'Immigrant Density (Mean)'
for i, row in enumerate(top_states.itertuples(), 1):
    top_states_table.cell(i, 0).text = str(row.us_state_enc)
    top_states_table.cell(i, 1).text = f"{row.import_from_slu_log:.4f}"
    top_states_table.cell(i, 2).text = f"{row.los_capped:.4f}"
    top_states_table.cell(i, 3).text = str(row.num_observations)
    top_states_table.cell(i, 4).text = f"{row.immigrant_density:.4f}"

In [ ]:
bottom_states = df_clean_nb.groupby('us_state_enc').agg({
    'import_from_slu_log': 'sum',
    'los_capped': 'mean',
    'us_state_enc': 'count',
    'immigrant_density': 'mean'
}).rename(columns={'us_state_enc': 'num_observations'}).reset_index()
bottom_states = bottom_states.sort_values(by='import_from_slu_log', ascending=True).head(10)
#print the bottom states table
print("\nBottom 10 States Importing from SLU:")
print(bottom_states)

# Add bottom states table to document
doc.add_paragraph('\n')
doc.add_heading('Bottom 10 States Importing from SLU', level=2)
bottom_states_table = doc.add_table(rows=len(bottom_states)+1, cols=5)
bottom_states_table.style = 'Table Grid'
bottom_states_table.cell(0, 0).text = 'State'
bottom_states_table.cell(0, 1).text = 'Total Import from SLU (Log)'
bottom_states_table.cell(0, 2).text = 'Mean Length of Stay (Capped)'
bottom_states_table.cell(0, 3).text = 'Number of Observations'
bottom_states_table.cell(0, 4).text = 'Immigrant Density (Mean)'
for i, row in enumerate(bottom_states.itertuples(), 1):
    bottom_states_table.cell(i, 0).text = str(row.us_state_enc)
    bottom_states_table.cell(i, 1).text = f"{row.import_from_slu_log:.4f}"
    bottom_states_table.cell(i, 2).text = f"{row.los_capped:.4f}"
    bottom_states_table.cell(i, 3).text = str(row.num_observations)
    bottom_states_table.cell(i, 4).text = f"{row.immigrant_density:.4f}"



In [ ]:
#create a table showing top 10 states with the highest average density and their LOS capped and number of observations
top_density_states = df_clean_nb.groupby('us_state_enc').agg({
    'immigrant_density': 'mean',
    'los_capped': 'mean',
    'us_state_enc': 'count',
    'import_from_slu_log': 'sum'
}).rename(columns={'us_state_enc': 'num_observations'}).reset_index()
top_density_states = top_density_states.sort_values(by='immigrant_density', ascending=False).head(10)
#print the top density states table
print("\nTop 10 States with Highest Average Density:")
print(top_density_states)
# Add top density states table to document
doc.add_paragraph('\n')
doc.add_heading('Top 10 States with Highest Average Density', level=2)
top_density_states_table = doc.add_table(rows=len(top_density_states)+1, cols=5)
top_density_states_table.style = 'Table Grid'
top_density_states_table.cell(0, 0).text = 'State'
top_density_states_table.cell(0, 1).text = 'Immigrant Density (Mean)'
top_density_states_table.cell(0, 2).text = 'Mean Length of Stay (Capped)'
top_density_states_table.cell(0, 3).text = 'Number of Observations'
top_density_states_table.cell(0, 4).text = 'Total Import from SLU (Log)'
for i, row in enumerate(top_density_states.itertuples(), 1):
    top_density_states_table.cell(i, 0).text = str(row.us_state_enc)
    top_density_states_table.cell(i, 1).text = f"{row.immigrant_density:.4f}"
    top_density_states_table.cell(i, 2).text = f"{row.los_capped:.4f}"
    top_density_states_table.cell(i, 3).text = str(row.num_observations)
    top_density_states_table.cell(i, 4).text = f"{row.import_from_slu_log:.4f}"      

In [ ]:
bottom_density_states = df_clean_nb.groupby('us_state_enc').agg({
    'immigrant_density': 'mean',
    'los_capped': 'mean',
    'us_state_enc': 'count',
    'import_from_slu_log': 'sum'
}).rename(columns={'us_state_enc': 'num_observations'}).reset_index()
bottom_density_states = bottom_density_states.sort_values(by='immigrant_density', ascending=True).head(10)
#print the bottom density states table
print("\nBottom 10 States with Highest Average Density:")
print(bottom_density_states)
# Add bottom density states table to document
doc.add_paragraph('\n')
doc.add_heading('Bottom 10 States with Highest Average Density', level=2)
bottom_density_states_table = doc.add_table(rows=len(bottom_density_states)+1, cols=5)
bottom_density_states_table.style = 'Table Grid'
bottom_density_states_table.cell(0, 0).text = 'State'
bottom_density_states_table.cell(0, 1).text = 'Immigrant Density (Mean)'
bottom_density_states_table.cell(0, 2).text = 'Mean Length of Stay (Capped)'
bottom_density_states_table.cell(0, 3).text = 'Number of Observations'
bottom_density_states_table.cell(0, 4).text = 'Total Import from SLU (Log)'
for i, row in enumerate(bottom_density_states.itertuples(), 1):
    bottom_density_states_table.cell(i, 0).text = str(row.us_state_enc)
    bottom_density_states_table.cell(i, 1).text = f"{row.immigrant_density:.4f}"
    bottom_density_states_table.cell(i, 2).text = f"{row.los_capped:.4f}"
    bottom_density_states_table.cell(i, 3).text = str(row.num_observations)
    bottom_density_states_table.cell(i, 4).text = f"{row.import_from_slu_log:.4f}"    



In [ ]:

# Create a table showing top 10 states by number of arrivals (observations), with their average LOS, immigrant density, and total imports from SLU
top_arrival_states = df_clean_nb.groupby('us_state_enc').agg({
    'los_capped': 'mean',
    'immigrant_density': 'mean',
    'import_from_slu_log': 'sum',
    'us_state_enc': 'count'
}).rename(columns={'us_state_enc': 'num_arrivals'}).reset_index()
top_arrival_states = top_arrival_states.sort_values(by='num_arrivals', ascending=False).head(10)

print("\nTop 10 States by Number of Arrivals:")
print(top_arrival_states)

# Add top arrival states table to document
doc.add_paragraph('\n')
doc.add_heading('Top 10 States by Number of Arrivals', level=2)
top_arrival_states_table = doc.add_table(rows=len(top_arrival_states)+1, cols=5)
top_arrival_states_table.style = 'Table Grid'
top_arrival_states_table.cell(0, 0).text = 'State'
top_arrival_states_table.cell(0, 1).text = 'Number of Arrivals'
top_arrival_states_table.cell(0, 2).text = 'Mean Length of Stay (Capped)'
top_arrival_states_table.cell(0, 3).text = 'Immigrant Density (Mean)'
top_arrival_states_table.cell(0, 4).text = 'Total Import from SLU (Log)'
for i, row in enumerate(top_arrival_states.itertuples(), 1):
    top_arrival_states_table.cell(i, 0).text = str(row.us_state_enc)
    top_arrival_states_table.cell(i, 1).text = str(row.num_arrivals)
    top_arrival_states_table.cell(i, 2).text = f"{row.los_capped:.4f}"
    top_arrival_states_table.cell(i, 3).text = f"{row.immigrant_density:.4f}"
    top_arrival_states_table.cell(i, 4).text = f"{row.import_from_slu_log:.4f}"

In [ ]:
#compare the average length of stay (LOS) capped between the top 10 states and bottom 10 states
top_avg_los = top_states['los_capped'].mean()
bottom_avg_los = bottom_states['los_capped'].mean()
print(f"\nAverage Length of Stay (Capped) - Top 10 States: {top_avg_los:.4f}")
print(f"Average Length of Stay (Capped) - Bottom 10 States: {bottom_avg_los:.4f}")

#compare the average length of stay (LOS) capped between the top 10 states by density and bottom 10 states by density
top_density_avg_los = top_density_states['los_capped'].mean()
bottom_density_avg_los = bottom_density_states['los_capped'].mean()
print(f"\nAverage Length of Stay (Capped) - Top 10 States by Density: {top_density_avg_los:.4f}")
print(f"Average Length of Stay (Capped) - Bottom 10 States by Density: {bottom_density_avg_los:.4f}")
print(f"\nAverage Length of Stay (Capped) - Top 10 States by Arrivals: {top_arrival_states['los_capped'].mean():.4f}")
print(f"Average Length of Stay (Capped) - Bottom 10 States by Arrivals: {bottom_states['los_capped'].mean():.4f}")

This prints all results into one word document

In [14]:
# Save the Word document
print("\nPlease select where to save the Word document...")
doc_path = select_file(
    "Save Analysis Report As", 
    [("Word Document", "*.docx"), ("All files", "*.*")],
    save=True
)

if doc_path:
    if not doc_path.endswith('.docx'):
        doc_path += '.docx'
    doc.save(doc_path)
    print(f"Analysis report saved to: {doc_path}")
else:
    print("Document not saved as no location was selected.")

print("\nAnalysis complete.")


Please select where to save the Word document...
Analysis report saved to: /Users/janai/Library/CloudStorage/OneDrive-SharedLibraries-jlconsulting.llc/Projects - Documents/Research/Saint Lucia Tourism Piece/30.06.2025 results with names.docx

Analysis complete.
